## Docker

**version**

In [ ]:
docker -v

**Extraer imagen mongodb**

In [ ]:
docker pull mongo

**Crear contenedor MongoDB**

In [ ]:
docker create -it --name MongoTest -p 5000:27017 mongo

**Iniciar contenedor**

In [ ]:
docker start MongoTest

**Detener contenedor**

In [ ]:
docker stop MongoTest

**Verificar**

In [ ]:
http://localhost:5000/

It looks like you are trying to access MongoDB over HTTP on the native driver port.

## Interacción con MongoDB

In [63]:
from flask import Flask, request, json, Response
from pymongo import MongoClient
import logging as log

In [69]:
class MongoAPI:
    def __init__(self, data):
        self.client = MongoClient("mongodb://localhost:27017/")  
        print(self.client)
        database = data['database']
        print(database)
        collection = data['collection']
        print(collection)
        cursor = self.client[database]
        print(cursor)
        self.collection = cursor[collection]
        self.data = data
        
        
    def read(self):
        documents = self.collection.find()
        print(documents)
        output = [{item: data[item] for item in data if item != '_id'} for data in documents]
        return output
    
    def write(self,data):
        log.info('Writing Data')
        new_document = data['Document']
        response = self.collection.insert_one(new_document)
        output = {'Status': 'Successfully Inserted',
                  'Document_ID': str(response.inserted_id)}
        return output
    
    def update(self):
        filt = self.data['Filter']
        updated_data = {"$set": self.data['DataToBeUpdated']}
        response = self.collection.update_one(filt, updated_data)
        output = {'Status': 'Successfully Updated' if response.modified_count > 0 else "Nothing was updated."}
        return output
    
    def delete(self, data):
        filt = data['Document']
        response = self.collection.delete_one(filt)
        output = {'Status': 'Successfully Deleted' if response.deleted_count > 0 else "Document not found."}
        return output

## Prueba

In [70]:
import json

In [71]:
data = {"database": "IshmeetDB","collection": "people",}
mongo_obj = MongoAPI(data)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
IshmeetDB
people
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'IshmeetDB')


In [72]:
print(json.dumps(mongo_obj.read(), indent=4))

[]


In [73]:
mongo_obj.read()

[]

## Flask

In [74]:
from flask import Flask, Response
app = Flask(__name__)

In [77]:
@app.route('/')
def base():
    return Response(response=json.dumps({"Status": "UP"}),
                    status=200,
                    mimetype='application/json')

app.run(debug=True, port=5001, host='0.0.0.0')

AssertionError: View function mapping is overwriting an existing endpoint function: base